In [1]:
import numpy as np
import pandas as pd
import pickle
import regex as re
from sklearn.model_selection import cross_val_predict

In [2]:
from preprocessing.parse_rs3_make_objects import EDUPair, EDU
from make_features import load_pairs_target_remove_minor_classes, generate_feature_matrix_, \
all_classification_reports, smote_oversampling, smote_oversampling5

In [3]:
# придется убрать классы, в которых меньше 3 объектов, а то все плохо
pairs,target = load_pairs_target_remove_minor_classes() 

In [4]:
X = generate_feature_matrix_(pairs)

(4672, 6235)


In [5]:
SEED=669

In [6]:
# oversampling 5 neighbors
X_res, target_res = smote_oversampling5(X, target) 

(48516, 6235)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target,
                                                    stratify=target, 
                                                    test_size=0.25, random_state=SEED)

In [8]:
X_train_res, y_train_res = smote_oversampling(X_train, y_train)

(36400, 6235)


In [9]:
SEED=669

In [10]:
## Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1500, random_state=SEED)

## Cross_val

In [13]:
pred_rf_cv = cross_val_predict(rf, X_res, target_res, cv=5, verbose=10)
all_classification_reports(target_res, pred_rf_cv)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 20.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 49.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 95.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 125.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 160.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 160.7min finished


All together:
                              precision    recall  f1-score   support

              attribution_NS       0.99      1.00      1.00      1866
              attribution_SN       0.98      0.99      0.98      1866
               background_NS       0.99      1.00      1.00      1866
               background_SN       1.00      1.00      1.00      1866
             cause-effect_NS       0.99      0.99      0.99      1866
             cause-effect_SN       1.00      0.99      1.00      1866
                comparison_M       1.00      1.00      1.00      1866
               concession_NS       0.99      1.00      1.00      1866
               concession_SN       1.00      1.00      1.00      1866
                condition_NS       0.99      1.00      0.99      1866
                condition_SN       0.99      1.00      0.99      1866
                  contrast_M       0.97      0.99      0.98      1866
              elaboration_NS       0.88      0.80      0.84      1866
     

## Train/test

In [11]:
rf.fit(X_train_res, y_train_res)
y_pred = rf.predict(X_test)

In [12]:
all_classification_reports(y_test, y_pred)

All together:
                              precision    recall  f1-score   support

              attribution_NS       1.00      0.09      0.17        11
              attribution_SN       0.48      0.32      0.38        38
               background_NS       0.00      0.00      0.00         9
               background_SN       0.00      0.00      0.00        10
             cause-effect_NS       1.00      0.15      0.26        20
             cause-effect_SN       1.00      0.10      0.17        21
                comparison_M       0.00      0.00      0.00        12
               concession_NS       0.00      0.00      0.00         7
               concession_SN       0.00      0.00      0.00         7
                condition_NS       0.00      0.00      0.00        10
                condition_SN       0.83      0.28      0.42        18
                  contrast_M       0.75      0.09      0.15        35
              elaboration_NS       0.49      0.16      0.24       181
     

/home/lena/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
